# Polars Tutorial - Part 7: Polars vs Pandas Comparison

In this final notebook, we'll compare Polars with Pandas:
- Performance benchmarks
- Memory usage comparison
- API differences and similarities
- Migration guide
- When to use which library

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import time
import os

DATA_DIR = '../data/'

print(f"Polars version: {pl.__version__}")
print(f"Pandas version: {pd.__version__}")

## 1. Performance Benchmarks

### 1.1 Reading CSV Files

In [ ]:
# Create a test CSV file
test_size = 500000
test_df = pl.DataFrame({
    'id': range(test_size),
    'value_a': np.random.randn(test_size),
    'value_b': np.random.randn(test_size),
    'category': np.random.choice(['A', 'B', 'C', 'D'], test_size),
    'date': ['2024-01-01'] * test_size
})
test_df.write_csv(os.path.join(DATA_DIR, 'benchmark.csv'))

print(f"Created test file with {test_size:,} rows")

In [ ]:
# Pandas read
start = time.time()
df_pandas = pd.read_csv(os.path.join(DATA_DIR, 'benchmark.csv'))
pandas_read_time = time.time() - start

# Polars read
start = time.time()
df_polars = pl.read_csv(os.path.join(DATA_DIR, 'benchmark.csv'))
polars_read_time = time.time() - start

print("CSV Reading Performance:")
print("=" * 50)
print(f"Pandas: {pandas_read_time:.4f} seconds")
print(f"Polars: {polars_read_time:.4f} seconds")
print(f"Speedup: {pandas_read_time/polars_read_time:.2f}x faster")

### 1.2 Filtering Operations

In [ ]:
# Pandas filtering
start = time.time()
pandas_filtered = df_pandas[(df_pandas['value_a'] > 0) & (df_pandas['category'] == 'A')]
pandas_filter_time = time.time() - start

# Polars filtering
start = time.time()
polars_filtered = df_polars.filter((pl.col('value_a') > 0) & (pl.col('category') == 'A'))
polars_filter_time = time.time() - start

print("Filtering Performance:")
print("=" * 50)
print(f"Pandas: {pandas_filter_time:.4f} seconds")
print(f"Polars: {polars_filter_time:.4f} seconds")
print(f"Speedup: {pandas_filter_time/polars_filter_time:.2f}x faster")

### 1.3 Group By Operations

In [ ]:
# Pandas group by
start = time.time()
pandas_grouped = df_pandas.groupby('category').agg({
    'value_a': ['sum', 'mean', 'std'],
    'value_b': ['sum', 'mean']
})
pandas_groupby_time = time.time() - start

# Polars group by
start = time.time()
polars_grouped = df_polars.group_by('category').agg([
    pl.sum('value_a').alias('value_a_sum'),
    pl.mean('value_a').alias('value_a_mean'),
    pl.std('value_a').alias('value_a_std'),
    pl.sum('value_b').alias('value_b_sum'),
    pl.mean('value_b').alias('value_b_mean')
])
polars_groupby_time = time.time() - start

print("Group By Performance:")
print("=" * 50)
print(f"Pandas: {pandas_groupby_time:.4f} seconds")
print(f"Polars: {polars_groupby_time:.4f} seconds")
print(f"Speedup: {pandas_groupby_time/polars_groupby_time:.2f}x faster")

### 1.4 Sorting Operations

In [ ]:
# Pandas sorting
start = time.time()
pandas_sorted = df_pandas.sort_values(['category', 'value_a'], ascending=[True, False])
pandas_sort_time = time.time() - start

# Polars sorting
start = time.time()
polars_sorted = df_polars.sort(['category', 'value_a'], descending=[False, True])
polars_sort_time = time.time() - start

print("Sorting Performance:")
print("=" * 50)
print(f"Pandas: {pandas_sort_time:.4f} seconds")
print(f"Polars: {polars_sort_time:.4f} seconds")
print(f"Speedup: {pandas_sort_time/polars_sort_time:.2f}x faster")

## 2. Memory Usage Comparison

In [ ]:
# Pandas memory usage
pandas_memory = df_pandas.memory_usage(deep=True).sum() / 1024 / 1024

# Polars memory usage
polars_memory = df_polars.estimated_size() / 1024 / 1024

print("Memory Usage Comparison:")
print("=" * 50)
print(f"Pandas: {pandas_memory:.2f} MB")
print(f"Polars: {polars_memory:.2f} MB")
print(f"Reduction: {(1 - polars_memory/pandas_memory) * 100:.1f}% less memory")

## 3. API Comparison

### 3.1 Creating DataFrames

In [ ]:
# Pandas
df_pd = pd.DataFrame({
    'A': [1, 2, 3],
    'B': ['a', 'b', 'c']
})

# Polars
df_pl = pl.DataFrame({
    'A': [1, 2, 3],
    'B': ['a', 'b', 'c']
})

print("Pandas DataFrame:")
print(df_pd)
print("\nPolars DataFrame:")
print(df_pl)

### 3.2 Selecting Columns

In [ ]:
# Pandas - multiple ways
print("Pandas column selection:")
print(df_pandas['category'].head(3))  # Single column as Series
print(df_pandas[['category', 'value_a']].head(3))  # Multiple columns

# Polars - explicit syntax
print("\nPolars column selection:")
print(df_polars['category'].head(3))  # Single column as Series
print(df_polars.select(['category', 'value_a']).head(3))  # Multiple columns

### 3.3 Filtering Rows

In [ ]:
# Pandas
pandas_result = df_pandas[df_pandas['value_a'] > 0]
print(f"Pandas filtered: {len(pandas_result)} rows")

# Polars
polars_result = df_polars.filter(pl.col('value_a') > 0)
print(f"Polars filtered: {polars_result.height} rows")

### 3.4 Adding Columns

In [ ]:
# Pandas - modifies in place (or creates copy)
df_pandas_copy = df_pandas.copy()
df_pandas_copy['sum_values'] = df_pandas_copy['value_a'] + df_pandas_copy['value_b']
print("Pandas with new column:")
print(df_pandas_copy[['value_a', 'value_b', 'sum_values']].head(3))

# Polars - returns new DataFrame (immutable)
df_polars_new = df_polars.with_columns(
    (pl.col('value_a') + pl.col('value_b')).alias('sum_values')
)
print("\nPolars with new column:")
print(df_polars_new.select(['value_a', 'value_b', 'sum_values']).head(3))

## 4. API Translation Guide

### Common Operations:

In [ ]:
operations_guide = pl.DataFrame({
    'Operation': [
        'Read CSV',
        'Filter rows',
        'Select columns',
        'Add column',
        'Group by',
        'Sort',
        'Drop duplicates',
        'Rename columns',
        'Fill null',
        'Join'
    ],
    'Pandas': [
        'pd.read_csv("file.csv")',
        'df[df["col"] > 5]',
        'df[["col1", "col2"]]',
        'df["new"] = df["a"] + df["b"]',
        'df.groupby("col").agg({...})',
        'df.sort_values("col")',
        'df.drop_duplicates()',
        'df.rename(columns={...})',
        'df.fillna(0)',
        'pd.merge(df1, df2, on="key")'
    ],
    'Polars': [
        'pl.read_csv("file.csv")',
        'df.filter(pl.col("col") > 5)',
        'df.select(["col1", "col2"])',
        'df.with_columns((pl.col("a") + pl.col("b")).alias("new"))',
        'df.group_by("col").agg([...])',
        'df.sort("col")',
        'df.unique()',
        'df.rename({...})',
        'df.fill_null(0)',
        'df1.join(df2, on="key")'
    ]
})

print("Pandas vs Polars API Translation:")
print(operations_guide.to_pandas().to_string(index=False))

## 5. Converting Between Polars and Pandas

In [ ]:
# Polars to Pandas
df_pl_small = pl.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
df_pd_from_pl = df_pl_small.to_pandas()

print("Polars to Pandas:")
print(type(df_pd_from_pl))
print(df_pd_from_pl)

# Pandas to Polars
df_pd_small = pd.DataFrame({'x': [7, 8, 9], 'y': [10, 11, 12]})
df_pl_from_pd = pl.from_pandas(df_pd_small)

print("\nPandas to Polars:")
print(type(df_pl_from_pd))
print(df_pl_from_pd)

## 6. Key Differences

### 6.1 Mutability

In [ ]:
# Pandas - mutable (can modify in place)
df_pd = pd.DataFrame({'a': [1, 2, 3]})
df_pd['b'] = [4, 5, 6]  # Modifies original
print("Pandas (mutable):")
print(df_pd)

# Polars - immutable (always returns new DataFrame)
df_pl = pl.DataFrame({'a': [1, 2, 3]})
df_pl_new = df_pl.with_columns(pl.lit([4, 5, 6]).alias('b'))  # Returns new DataFrame
print("\nPolars original (unchanged):")
print(df_pl)
print("\nPolars new (with column added):")
print(df_pl_new)

### 6.2 Expression System

In [ ]:
# Pandas - direct column access
df_pd = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
df_pd['c'] = df_pd['a'] * 2 + df_pd['b']

# Polars - expression-based
df_pl = pl.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
df_pl_result = df_pl.with_columns(
    (pl.col('a') * 2 + pl.col('b')).alias('c')
)

print("Pandas result:")
print(df_pd)
print("\nPolars result:")
print(df_pl_result)

## 7. When to Use Which?

### Use Polars When:
- ✅ Working with large datasets (> 1GB)
- ✅ Performance is critical
- ✅ Memory efficiency matters
- ✅ You need parallel processing
- ✅ Complex ETL pipelines
- ✅ Data warehouse-style operations

### Use Pandas When:
- ✅ Working with small datasets (< 1GB)
- ✅ Need compatibility with existing code
- ✅ Using libraries that require Pandas
- ✅ Interactive data exploration
- ✅ Time series analysis (more mature support)
- ✅ Need mutable DataFrames

## 8. Migration Strategy

In [ ]:
# Example migration workflow

# Step 1: Read with Polars for performance
df_polars = pl.read_csv(os.path.join(DATA_DIR, 'sales_data.csv'))

# Step 2: Do heavy processing in Polars
processed = df_polars.filter(
    pl.col('revenue') > 500
).group_by('category').agg([
    pl.sum('revenue').alias('total_revenue'),
    pl.len().alias('count')
])

# Step 3: Convert to Pandas for visualization or library compatibility
df_for_viz = processed.to_pandas()

print("Hybrid approach (Polars for processing, Pandas for viz):")
print(df_for_viz)

# Visualization would go here
# import matplotlib.pyplot as plt
# df_for_viz.plot.bar(x='category', y='total_revenue')

## 9. Complete Performance Summary

In [ ]:
# Summary of all benchmarks
benchmark_results = pl.DataFrame({
    'Operation': ['Read CSV', 'Filter', 'GroupBy', 'Sort'],
    'Pandas_Time': [pandas_read_time, pandas_filter_time, pandas_groupby_time, pandas_sort_time],
    'Polars_Time': [polars_read_time, polars_filter_time, polars_groupby_time, polars_sort_time]
}).with_columns(
    (pl.col('Pandas_Time') / pl.col('Polars_Time')).alias('Speedup')
)

print("\nPerformance Summary (500,000 rows):")
print("=" * 70)
print(benchmark_results.to_pandas().to_string(index=False))

avg_speedup = benchmark_results['Speedup'].mean()
print(f"\nAverage Speedup: {avg_speedup:.2f}x faster")

## 10. Summary

In this notebook, we compared Polars with Pandas:
- ✅ Performance benchmarks across common operations
- ✅ Memory usage comparison
- ✅ API differences and translation guide
- ✅ Converting between the libraries
- ✅ Key philosophical differences
- ✅ When to use which library
- ✅ Migration strategies

### Key Findings:
1. **Polars is significantly faster** - Often 5-50x faster than Pandas
2. **Lower memory usage** - Polars uses less memory for the same data
3. **Different philosophy** - Polars is immutable and expression-based
4. **Easy conversion** - Can easily move between Polars and Pandas
5. **Complementary** - Can use both in the same workflow

### Polars Advantages:
- ⚡ Much faster for large datasets
- 💾 More memory efficient
- 🔄 Automatic parallelization
- 🎯 Query optimization via lazy evaluation
- 🛡️ Type safety and immutability
- 📊 Better for data engineering tasks

### Pandas Advantages:
- 🔧 More mature ecosystem
- 📚 Extensive documentation and community
- 🔌 Better library integration
- 📈 More features for time series
- 🎨 More flexible for exploration
- 🔄 Mutable (can be advantage or disadvantage)

### Recommendation:
- Start new projects with **Polars** if performance matters
- Use **Pandas** for quick exploration and when you need specific libraries
- Consider a **hybrid approach**: Polars for ETL, Pandas for analysis/viz
- Both libraries can coexist in the same project!

---

## Congratulations! 🎉

You've completed the Polars tutorial series! You now know:
- How to use Polars for data manipulation
- How to leverage parallel processing
- How to optimize queries with lazy evaluation
- When to use Polars vs Pandas
- How to create visualizations with Polars data

**Keep exploring and happy data processing with Polars!** 🚀